# *Fixed Income and Credit Master in Computational Finance (MCF) - Final Exam* 

### **1 Yield Curve Fitting (15 pts)**

The Nelson–Siegel model is widely used in practice for yield curve fitting. It postulates that the term structure can be described by
<br>
$$r(0, t) = \beta_{0} + \beta_{1}\frac{1 − e^{−t/\theta}}{t/\theta} + \beta_{2}(\frac{1 − e^{−t/\theta}}{t/\theta} − e^{−t/\theta})$$
where $r_{0,t}$ is the spot rate for maturity $t, \beta_{0}, \beta_{1}$, and $\beta_{2}$ represent the level, slope, and curvature parameter, and $θ$ is the time scaling parameter. The model assumes continuous compounding.

(a) **(1pts)** Show that the instantaneous forward rate at future date t can be calculated as : 
$$f_t = r(0, t) + t \cdot \frac{dr(0, t)}{dt}$$
<u>_Hint_</u>: The instantaneous forward rate is defied as
$$f_t := \lim_{\delta \to 0} f_{t,t+\delta t}$$

**Proof:**

From $r(0,t) = \frac{1}{t}\int_0^t f(0,t')dt'$ and by directly differentiating $\frac{d}{dt} r(0,t)$, the RHS of the statement becomes

\begin{equation*}
r(0, t) + t \frac{dr(0, t)}{dt} = \frac{d}{dt} \left( \int_0^{t}f(0, t') dt' \right)
\end{equation*}
From the definition of the first derivative we get
\begin{align*}
\frac{d}{dt} \left( \int_0^{t}f(0, t') dt' \right) &=  \lim_{\delta t \to 0} \frac{1}{\delta t} \left[ \int_0^{t+\delta t}f(0, t') dt' - \int_0^{t}f(0, t') dt' \right] \\
&= \lim_{\delta t \to 0} \frac{1}{\delta t} \left[ \int_t^{t+\delta t}f(0, t') dt' \right]
\end{align*}

It is safe to assume that $f(t_1, t_2)$ can be approximated by its average on a small interval $[t_1, t_2]$. This approximation becomes more and more precise as $t_2 \to t_1$. In other words:

\begin{equation*}
\lim_{\delta t \to 0} f(t, t + \delta t) = \lim_{\delta t \to 0} \frac{1}{\delta t} \int_t^{t+\delta t}f(t, t') dt'
\end{equation*}

The last step would be to prove

\begin{equation*}
\lim_{\delta t \to 0} \frac{1}{\delta t} \int_t^{t+\delta t}f(t, t') dt' = \lim_{\delta t \to 0} \frac{1}{\delta t} \int_t^{t+\delta t}f(0, t') dt'
\end{equation*}

Now we need either the definition of $f(t_1,t_2)$ or an argument why the last equation holds. The last equation can be justified by following argumentation: the dependency of forward rates on the length of the interval $t_2-t_1$ is much stronger that its dependency on the individual values of $t_1$ and $t_2$ (i.e. where in time the interval is located). In other words, it is more important for how long we are going to hold forward rate once it starts paying the cash flow than then in what exact period of time we are going to do it. This approximation is probably more accurate for more flat yield curves, and more accurate as the time interval becomes smaller.

Side note: if we had an explicit definition of $f(t_1,t_2)$, or even a $\frac{\partial }{\partial t_1}f(t_1, t_2)$ we could use Leibniz integral rule (derivative of an integral with variable limits of integration) and prove it explicitly.

(b) **(1pts)** Using the above result, show that the instantaneous forward rate in the Nelson–Siegel model is
$$ f_t = \beta_{0} + \beta_{1}e^{−t/\theta} + \beta_{2}\frac{e^{−t/\theta}}{\theta/t}$$

**Proof** This is straight forward differentiation of $\frac{d}{dt}(r(0,t))$ for Nelson–Siegel model. We will let sympy do that for us:

In [1]:
import sympy as sp

# defining the symbols
b0, b1, b2, t, theta = sp.symbols('b0,b1,b2,t,theta')

# defining the NS model for term structure
r = b0 + b1*((1-sp.exp(-t/theta))/(t/theta)) + b2*((1-sp.exp(-t/theta))/(t/theta)-sp.exp(-t/theta))

# simplfy r+tr' using r from NS model
(r + t*sp.diff(r, t)).simplify()

b0 + b1*exp(-t/theta) + b2*t*exp(-t/theta)/theta

(c) **(7 pts)** Consider the market spot rate curve provided in the Excel sheet Yield Curve. For each given date, fit the Nelson–Siegel function to the market data. Create a table which reports $\hat{\beta_{0}}, \hat{\beta_{1}}, \hat{\beta_{2}}, \hat{\theta}$, as well as the mean squared error (MSE), i.e., the row names should be observation dates, whereas the column dates should be the model coefficients.
<br>
<br>
<u>_Hint_</u>: There are two approaches to this problem. First, you can run an optimization algorithm to fit the Nelson –Siegel function to market data on each observation date (e.g., in Python, you can use scipy.optimize). Alternatively, you can cast the problem into an ordinary least squares (OLS) regression framework by creating a grid of θ-values (e.g., ranging from 1 to 10, as suggested in the literature, however you can also try larger values if necessary). For each observation date, pick one θ-value, run the OLS regressions, and record the estimated coefficients and MSE. Iterate this
procedure over the whole θ-grid, and select the final solution based on the optimal θ. Move to the next observation date and repeat the procedure. The final table should contain only your selected output, not all intermediary results.

In [2]:
#importing the necessary packages
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import math
import scipy.optimize as sco



#defining the Nelson-Siegel function
def Nelson(b0,b1,b2,t,teta):
    return b0+b1*((1-math.e**(-t/teta))/(t/teta))+b2*(((1-math.e**(-t/teta))/(t/teta))-math.e**(-t/teta))

#importing the file
imported_xlsx = pd.read_excel('MCF22_FIC_Final_Exam_Group_7.xlsx')
df = imported_xlsx.iloc[:,1:]

#performing the optimization
#due to the computation problems we have  run computation in 2 parts

#first 45 rows
parametri_45=[]
msfe_45=[]
sucess_check_45=[]#all optimizitations should have success true
np.set_printoptions(suppress=True)

for i in range(45):
    parametri_45.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[2]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['x'])
    msfe_45.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[2]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['fun'])
    sucess_check_45.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[2]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['success'])

#second half from 45 to 189    

parametri_45_189=[]
msfe_45_189=[]
sucess_check_45_189=[] #all optimizitations should have success true

for i in range(45,197):
    parametri_45_189.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[1]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['x'])
    msfe_45_189.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[1]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['fun'])
    sucess_check_45_189.append(sco.minimize(lambda parametri:sum((df.iloc[i,:].values-Nelson(parametri[0],parametri[1],parametri[2],np.array(df.columns),parametri[3]))**2),[1]*4,constraints=dict(type='ineq',fun=lambda parametri:parametri[0]))['success'])
    
    
#creating data frame
df_rad=pd.DataFrame(np.concatenate((parametri_45,parametri_45_189)))
df_rad['MSFE']=np.concatenate((msfe_45,msfe_45_189))
df_rad['MSFE_da_bude_isto']=df_rad['MSFE']/16  #MSFE is the total squared variation, while MSFE_da_bude isto is just the average squared variation
df_rad.index=pd.read_excel('MCF22_FIC_Final_Exam_Group_7.xlsx').iloc[:,0]
df_rad.columns=['b0','b1','b2','teta','MSFE','MSFE_da_bude_isto']

In [3]:
df_rad

,b0,b1,b2,teta,MSFE,MSFE_da_bude_isto
Date,,,,,,
2005-01-31,5.096785,-2.576285,-0.216951,3.161346,0.147391,0.009212
2005-02-28,5.038708,-2.273468,-0.002083,2.568400,0.149450,0.009341
2005-03-31,5.016114,-2.205774,0.000287,1.929730,0.209139,0.013071
2005-04-29,4.876309,-1.869713,-0.003075,3.250367,0.168928,0.010558
2005-05-31,4.665115,-1.618400,-0.001917,3.767616,0.160179,0.010011
...,...,...,...,...,...,...
2021-01-29,2.503868,-2.406583,-2.993168,2.798184,0.004617,0.000289
2021-02-26,2.753512,-2.666360,-3.327143,2.101315,0.003281,0.000205
2021-03-31,2.975027,-2.847894,-4.046063,1.706021,0.014094,0.000881


(d) **(6 pts)**  Create the following six graphs:
* A chart which shows the time evolution of the MSE,
* A chart which jointly shows the time evolution of the coefficients $\hat{\beta_{0}}, \hat{\beta_{1}}, \hat{\beta_{2}}$,
* Three charts which draw the fitted curves and the corresponding market spot
rates on the dates when your model produced best, worst, and medium quality
results in terms of the MSE.

In [4]:
#evolution of the MSE over time

import plotly.graph_objects as go
fig1=go.Figure()

fig1.add_trace(go.Scatter(y=df_rad['MSFE_da_bude_isto'],x=df_rad.index))
fig1.update_layout(
    title="MSE evolution",
    xaxis_title="Date",
    yaxis_title="MSE")

In [5]:
# Beta0,Beta1,Beta2 and theta over time

fig=go.Figure()

fig.add_trace(go.Scatter(y=df_rad['b0'],x=df_rad.index,name=r'$\beta_{0}$'))
fig.add_trace(go.Scatter(y=df_rad['b1'],x=df_rad.index,name=r'$\beta_{1}$'))
fig.add_trace(go.Scatter(y=df_rad['b2'],x=df_rad.index,name=r'$\beta_{2}$'))
fig.add_trace(go.Scatter(y=df_rad['teta'],x=df_rad.index,name=r'$\theta$'))

fig.update_layout(
    title="Evolution of parameters",
    xaxis_title="Date",
    yaxis_title="Parameter value")

In [6]:
#plotting the best fit

df_sa_datumom=imported_xlsx
maturities = np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30])

fig4=go.Figure()

fig4.add_trace(go.Scatter(y=Nelson(b0=df_rad.sort_values('MSFE').iloc[0,0:4][0],b1=df_rad.sort_values('MSFE').iloc[0,0:4][1],b2=df_rad.sort_values('MSFE').iloc[0,0:4][2],t=np.array(df.columns),teta=df_rad.sort_values('MSFE').iloc[0,0:4][3]), x=maturities, name='Fitted'))
fig4.add_trace(go.Scatter(y=df_sa_datumom[df_sa_datumom['Date']=='2020-09-30'].values[0][1:], x=maturities, name='Actual'))

fig4.update_layout(
    title="The best fit",
    xaxis_title="Time to maturity [years]",
    yaxis_title="Interest rate [%]")

In [7]:
#the worst fit

fig3=go.Figure()

fig3.add_trace(go.Scatter(y=Nelson(b0=df_rad.sort_values('MSFE').iloc[196,0:4][0],b1=df_rad.sort_values('MSFE').iloc[196,0:4][1],b2=df_rad.sort_values('MSFE').iloc[196,0:4][2],t=np.array(df.columns),teta=df_rad.sort_values('MSFE').iloc[196,0:4][3]), x=maturities, name='Fitted'))

fig3.add_trace(go.Scatter(y=df_sa_datumom[df_sa_datumom['Date']=='2008-10-31'].values[0][1:], x=maturities, name='Actual'))

fig3.update_layout(
    title="The Worst fit",
    xaxis_title="Time to maturity [years]",
    yaxis_title="Interest rate [%]")

In [8]:
#averaged fit 

fig5=go.Figure()

fig5.add_trace(go.Scatter(y=Nelson(b0=df_rad.sort_values('MSFE').iloc[int(196/2),0:4][0],b1=df_rad.sort_values('MSFE').iloc[int(196/2),0:4][1],b2=df_rad.sort_values('MSFE').iloc[int(196/2),0:4][2],t=np.array(df.columns),teta=df_rad.sort_values('MSFE').iloc[int(196/2),0:4][3]), x=maturities, name='Fitted'))
fig5.add_trace(go.Scatter(y=df_sa_datumom[df_sa_datumom['Date']=='2010-11-30'].values[0][1:], x=maturities, name='Actual'))

fig5.update_layout(
    title="Medium fit",
    xaxis_title="Time to maturity [years]",
    yaxis_title="Interest rate [%]")

### **2 Principal Component Analysis (9 pts)**

Principal component analysis (PCA) is a statistical technique commonly used in finance to reduce the dimensionality of the data set. This is particularly important for strongly correlated systems such as yield curves, where multicollinearity issues are pervasive.
<br>
Consider the collection of (continuously compounded) spot rates provided in the Excel sheet **PCA** as well as the full set of market spot rates provided in the Excel sheet **Yield Curve**.

(a) **(5 pts)** Run a PCA on the covariance matrix of the spot rates for both data sets. For each data set, report the fraction of variance explained by each principal component. Is there one dominant component? Discuss your findings. 
<p> <u><i>Hint</i></u>: Running a PCA boil down to an eigenvalue decomposition. For example, you can use <b>numpy.linalg.eig</b> in Python.</p>

In [9]:
import numpy as np
from numpy import linalg as LA

In [10]:
import pandas as pd
df1 = pd.read_excel("MCF22_FIC_Final_Exam_Group_7.xlsx",sheet_name='Yield_Curve',index_col="Date")
PCA = pd.read_excel("MCF22_FIC_Final_Exam_Group_7.xlsx",sheet_name='PCA',index_col="Date")
PCA

,1Y,2Y,3Y,4Y,5Y
Date,,,,,
2005-01-31,2.9727,3.2573,3.4251,3.5557,3.6788
2005-02-28,3.2540,3.5845,3.7567,3.8734,3.9757
2005-03-31,3.4287,3.7855,3.9514,4.0541,4.1429
2005-04-29,3.3855,3.6232,3.7250,3.7923,3.8608
2005-05-31,3.3845,3.5692,3.6303,3.6672,3.7129
...,...,...,...,...,...
2021-01-29,0.0998,0.1132,0.2025,0.3317,0.4779
2021-02-26,0.0956,0.1654,0.3433,0.5569,0.7704
2021-03-31,0.0747,0.1683,0.3967,0.6694,0.9395


In [11]:
eig_vals_PCA, eig_vec_PCA = LA.eig(np.cov(PCA.T))#computing eigenvalues and eigenvectors for PCA dataset
eig_vals_df, eig_vec_df = LA.eig(np.cov(df1.T))#computing eigenvalues and eigenvectors for yield dataset 

In [12]:
eig_vec_PCA

array([[ 0.50097219, -0.61329362, -0.56111974,  0.23565815, -0.05007669],
       [ 0.47731744, -0.2690365 ,  0.41642263, -0.65870333,  0.30412092],
       [ 0.4459101 ,  0.07354695,  0.50461651,  0.30251443, -0.67052384],
       [ 0.41503519,  0.37782821,  0.11682936,  0.51464652,  0.63755901],
       [ 0.38743373,  0.6350803 , -0.49340643, -0.39268198, -0.22117657]])

In [13]:
lambda2 = eig_vals_PCA / sum(eig_vals_PCA)#fraction of variance for PCA dataset
lambda2

array([0.97884847, 0.02045006, 0.0006851 , 0.0000163 , 0.00000007])

**comment:** First component explains almost everything (97.9% of variance). 

In [14]:
lambda1=eig_vals_df/sum(eig_vals_df) #fraction of variance for yield curve dataset
lambda1

array([0.86187021, 0.12675215, 0.00848067, 0.00190311, 0.00075901,
       0.00019012, 0.00003322, 0.00001024, 0.00000118, 0.00000008,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

**comment:** First component explains about 86\%. By adding the second component we are able to explain almost everything (98.9% of variance).

(b) **(2 pts)** Plot the loadings for each factor for both data sets. What can you say about the loadings for the three most dominant factors? 

In [15]:
#PCA
maturities_pca_set = [1,2,3,4,5]
fig1=go.Figure()
fig1.add_trace(go.Scatter(y=eig_vec_PCA[:,0],x=maturities_pca_set, name=r'Loading for $PC_{1}$'))
fig1.add_trace(go.Scatter(y=eig_vec_PCA[:,1],x=maturities_pca_set, name=r'Loading for $PC_{2}$'))
fig1.add_trace(go.Scatter(y=eig_vec_PCA[:,2],x=maturities_pca_set, name=r'Loading for $PC_{3}$'))

fig1.update_layout(
    autosize=False,
    title="Loadings of 'PCA' data set",
    xaxis_title="Maturity [years]",
    yaxis_title="Loadings")

The first component captures the overall level of the yield curve. The second component captures the slope of the yield curve because it has bigger loadings on longer maturities, almost perfectly linearly increasing loadings as maturities increase. For the third PC we see that it captures curvature. 

In [16]:
# df loadings

maturities = np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30])

fig1=go.Figure()
fig1.add_trace(go.Scatter(y=eig_vec_df[:,0],x=maturities,name=r'Loading for $PC_{1}$'))
fig1.add_trace(go.Scatter(y=eig_vec_df[:,1],x=maturities,name=r'Loading for $PC_{2}$'))
fig1.add_trace(go.Scatter(y=eig_vec_df[:,2],x=maturities,name=r'Loading for $PC_{3}$'))
fig1.update_layout(
    autosize=False,
    title="Loadings of 'Yield_curve' data set",
    xaxis_title="Maturity [years]",
    yaxis_title="Loadings")

The first component captures the overall level and to some extent the slope of the yield curve.

The second component captures curvature, and probably the near and mid term slopes of the yield curves as well as the long-term level.

The third component captures the mid-term hump at 5-years maturity. We did not have this kind of component in the "PCA" data set, maybe because it does not have many curves with pronounced humps, or the humps are not large enough to be significant and then represented by one of the first three PCs. 

(c) **(2 pts)** Plot the three most dominant factors over time for both data sets. What do these two factors resemble and how do they compare between the two data sets? 

In [17]:
#PCA data set
fig=go.Figure()
fig.add_trace(go.Scatter(y=((LA.inv(eig_vec_PCA)@PCA.T).T).iloc[:,0],x=PCA.index, name='First component (level)'))
fig.add_trace(go.Scatter(y=((LA.inv(eig_vec_PCA)@PCA.T).T).iloc[:,1],x=PCA.index, name='Second component (slope)'))
fig.add_trace(go.Scatter(y=((LA.inv(eig_vec_PCA)@PCA.T).T).iloc[:,2],x=PCA.index, name='Third component (curvature)'))

In [18]:
#Yield Curve dataset

fig1=go.Figure()
fig1.add_trace(go.Scatter(y=-((LA.inv(eig_vec_df)@df.T).T).iloc[:,0], name='First component (level; changed sign)'))
fig1.add_trace(go.Scatter(y=((LA.inv(eig_vec_df)@df.T).T).iloc[:,1], name='Second component (near-term yield)'))
fig1.add_trace(go.Scatter(y=((LA.inv(eig_vec_df)@df.T).T).iloc[:,2], name='Third component (5-year hump) '))

We intentionally plotted the first component of the  "Yield Curve" dataset in order to compare it to the "PCA" set. The first component (level) has similar behavior in both data sets, and both levels dropped after the GFC (2008) and COVID crisis. We see that the near-term component rises during and a few years after the crisis, and it happens as bonds drop in value. Evolution of the third component does not have obvious interpretation on any of two data sets.

### **3 Bond Returns Predictability (18 pts)**

In this exercise, we will study the expectations hypothesis. In particular, we will consider the famous Fama–Bliss model and try to answer some questions regarding the bond returns predictability. Use the data set provided in in the Excel sheet **Fama Bliss**.
<p><u><i>Remark</u>:</i> Please note that in this exercise we adopt the following notation: 

*Subscripts* represent the point in time (measured in years), and *superscripts* (parenthesized) denote the bond maturity/tenor (also measured in years). Read carefully through other notations and conventions introduced below.</p>

(a) **(3 pts)** Using the available zero-coupon bond prices $P_{t}^{(n)}$ (n = 1, 2, . . . , 5), construct continuously compounded yields $y_{t}^{(n)}$. 
<br>
Produce a table in which you report the following statistics:

*   The average yield $\mathbb{E}[y_{t}^{(n)}]$,
*   The yield volatility ${\sigma}[y_{t}^{(n)}]$,
*   The average term spread $\mathbb{E}[y_{t}^{(n)} - y_{t}^{(1)}]$,
*   The average excess return $\mathbb{E}[r_{t+1}^{(n)}- y_{t}^{(1)}]$, where $r_{t+1}^{(n)}:= ln(P_{t+1}^{(n-1)} / P_{t}^{(n)})$,
*   The excess return volatility ${\sigma}[r_{t+1}^{(n)} - y_{t}^{(1)}]$,
*   The implied Sharpe ratio,
<br>
where $\mathbb{E}[·]$ and $σ(·)$ represent the unconditional expectation and volatility, respectively. Is the average risk premium for long-term bonds relatively high or low? Why would you care to hold long-term bonds in your portfolio? Justify your answer.

In [19]:
import pandas as pd
import numpy as np

prices=pd.read_excel('MCF22_FIC_Final_Exam_Group_7.xlsx',sheet_name='Fama_Bliss',index_col='Date')#importing the dataset
prices

,1Y,2Y,3Y,4Y,5Y
Date,,,,,
1964-01-31,96.29,92.50,88.85,85.40,81.92
1964-02-28,96.14,92.34,88.62,85.16,81.76
1964-03-31,96.17,92.17,88.36,84.73,81.59
1964-04-30,96.29,92.36,88.59,84.93,81.55
1964-05-28,96.24,92.43,88.76,85.10,81.94
...,...,...,...,...,...
2020-08-31,99.86,99.73,99.54,99.15,98.63
2020-09-30,99.87,99.75,99.51,99.16,98.59
2020-10-30,99.87,99.69,99.40,98.81,98.03


We will divide all prices by 100 in order to follow the notation form lectures. This way we assume the face values of these bonds are \$100.

In [20]:
y=-np.log(prices/100)/np.arange(1,6) #calculating the continuously compounded yields
y

,1Y,2Y,3Y,4Y,5Y
Date,,,,,
1964-01-31,0.037806,0.038981,0.039407,0.039456,0.039885
1964-02-28,0.039365,0.039846,0.040271,0.040160,0.040276
1964-03-31,0.039053,0.040768,0.041250,0.041425,0.040693
1964-04-30,0.037806,0.039738,0.040384,0.040836,0.040791
1964-05-28,0.038325,0.039359,0.039745,0.040336,0.039837
...,...,...,...,...,...
2020-08-31,0.001401,0.001352,0.001537,0.002134,0.002759
2020-09-30,0.001301,0.001252,0.001637,0.002109,0.002840
2020-10-30,0.001301,0.001552,0.002006,0.002993,0.003979


In [21]:
#The average yield E[y(n)t]
np.mean(y,axis=0)

1Y    0.050169
2Y    0.052191
3Y    0.053999
4Y    0.055610
5Y    0.056734
dtype: float64

In [22]:
#The yield volatility σ[y(n)t]
np.std(y,axis=0)

1Y    0.033481
2Y    0.033076
3Y    0.032356
4Y    0.031686
5Y    0.030984
dtype: float64

In [23]:
#The average term spread
avg=[]
for i in range(1,5):
    avg.append([y.iloc[:,i]-y.iloc[:,i-1]])
    
[np.mean(i) for i in avg]#The average term spread

[0.002021381401215052,
 0.0018087398841606366,
 0.0016111777130120432,
 0.0011237665586219165]

In [24]:
# in order to do stuff with the excess return we must first compute the necessary variables
rx_buffer=[] #list for storage of values
for i in range(4):
    rx_buffer.append([np.log(prices.iloc[12:,i].values / prices.iloc[:-12,i+1].values)])

#putting the data in the data frame
rx=pd.DataFrame(rx_buffer[0][0])

rx['dvojka']=pd.DataFrame(rx_buffer[1][0])
rx['trojka']=pd.DataFrame(rx_buffer[2][0])
rx['stiri']=pd.DataFrame(rx_buffer[3][0])

#computing the necessary varibles
avg=[np.mean(rx.iloc[:,i].values-y.iloc[:-12,i].values) for i in range(4)] 

#The average excess return
avg

[0.004752829252600009,
 0.006784487945831464,
 0.008466522443895072,
 0.008248099149635554]

In [25]:
std=[np.std(rx.iloc[:,i].values-y.iloc[:-12,i].values) for i in range(4)]
#The excess return volatility
std

[0.016844319074390372,
 0.030051296436156866,
 0.04148540730115075,
 0.05117126196471423]

In [26]:
for i,j in zip(avg,std):
    print(i/j)  #implied Sharpe ratio

0.2821621480577435
0.2257635693103929
0.20408435145483608
0.16118615865528452


**comment**  The average risk premium for long-term bonds is relatively low. Sharpe ratio is lower for long-term bonds, mainly because the volatility of excess returns rises sharply with increase in maturity. Holding long term bonds is useful if we do not need money in near term (pension plan for example). We can buy long term bonds after the crisis period when yields are high, expecting the price of the bond to rise as economy recovers. Also, being short in long term bonds can be used for hedging the short and mid term bonds against the interest rate risk, which is another reason for having long term bonds in portfolio (although with negative weights).

(b) In the lecture notes, we provided results based on the following (one-step-ahead) predictive regression analysis for n-year zero-coupon bonds (n = 2, 3, 4, 5) for the period 1964M1–2021M05:
$$rx_{t+1}^{(n)} = a^{(n)} + b^{(n)}(f_{t}^{(n)} − y_{t}^{(1)}) + ε_{t+1}^{(n)},$$
where $rx_{t+1}^{(n)}:=r_{t+1}^{(n)} − y_{t}^{(1)}$ is the excess bond return over the period $[t, t+1]$, and $f_{t}^{(n)}$ is the forward rate for the period $[t + n − 1, t + n]$.
>(b.1)  **(6 pts)** Run the above regression for the following periods: 1964M1–1973M12, 1974M1–1983M12, 1984M1–1993M12, 1994M1–2003M12, 2004M1–2013M12,
and 2014M1–2021M05. In your output tables, report any statistics you consider
important for a regression analysis. How does the predictability change over time? How does the predictability change over maturities? Discuss your
results.

In [27]:
#calculating forward rates

forward_rates=[]
for i in range(4):
    forward_rates.append([np.log(prices.iloc[:,0].values)-np.log(prices.iloc[:,1].values)])
    
forward_rates_df=pd.DataFrame(forward_rates[0][0])
forward_rates_df[1]=pd.DataFrame(forward_rates[1][0])
forward_rates_df[2]=pd.DataFrame(forward_rates[2][0])
forward_rates_df[3]=pd.DataFrame(forward_rates[3][0])
forward_rates_df

,0,1,2,3
0,0.040156,0.040156,0.040156,0.040156
1,0.040328,0.040328,0.040328,0.040328
2,0.042483,0.042483,0.042483,0.042483
3,0.041670,0.041670,0.041670,0.041670
4,0.040393,0.040393,0.040393,0.040393
...,...,...,...,...
679,0.001303,0.001303,0.001303,0.001303
680,0.001202,0.001202,0.001202,0.001202
681,0.001804,0.001804,0.001804,0.001804
682,0.001804,0.001804,0.001804,0.001804


In [28]:
# importing necessary packages

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

/home/miroslav/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/miroslav/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning:

pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



In [29]:
step = 120
tables = []

for col in range(0,4):
  table = []
  for row in range(1,rx.shape[0]//step+1):
    Y11=rx.iloc[(row-1)*step:row*step,col].values-y.iloc[(row-1)*step:row*step,col].values
    X11=forward_rates_df.iloc[(row-1)*step:row*step,col].values-y.iloc[(row-1)*step:row*step,col].values

    lm = LinearRegression()
    lm.fit(X11.reshape(X11.shape[0],1), Y11)
    res1 = (lm.intercept_, lm.coef_[0], lm.score(X11.reshape(X11.shape[0],1), Y11))
    table.append(np.array(res1))
    #print(row*step, row)
  
  last = (rx.shape[0]//step)*step
  Y11=rx.iloc[last:last+(rx.shape[0]%step),col].values-y.iloc[last:last+(rx.shape[0]%step),col].values
  X11=forward_rates_df.iloc[last:last+(rx.shape[0]%step),col].values-y.iloc[last:last+(rx.shape[0]%step),col].values

  lm = LinearRegression()
  lm.fit(X11.reshape(X11.shape[0],1), Y11)
  res1 = (lm.intercept_, lm.coef_[0], lm.score(X11.reshape(X11.shape[0],1), Y11))
  table.append(np.array(res1))

  tables.append(np.array(table))

dfs = [pd.DataFrame(table, columns=['intercept', 'b', 'r_sqared']) for table in tables]
for df in dfs:
  print(df) #four tables where each of them represents maturity (n = 2, 3, 4, 5). Rows represents subintervals in such a way that 0 (first 120 obs), 1(from 120 obs to 240) and so on

   intercept         b  r_sqared
0  -0.004152  0.820967  0.119412
1  -0.001295  0.556941  0.046484
2   0.003907  1.148323  0.141738
3   0.006837  0.425780  0.037186
4   0.006502 -0.461208  0.052452
5   0.005942 -0.816015  0.135232
   intercept         b  r_sqared
0  -0.004738  1.907086  0.058260
1  -0.007431  1.115914  0.014794
2   0.004600  3.392451  0.092192
3   0.010108  1.454501  0.029110
4   0.014419 -2.414246  0.093965
5   0.012668 -3.685728  0.153525
   intercept          b  r_sqared
0  -0.007325   2.119684  0.020804
1  -0.009751  -0.165393  0.000074
2   0.014180   8.665775  0.123629
3   0.015577   4.036864  0.050169
4   0.009603 -10.264363  0.214987
5   0.010093  -4.409736  0.025699
   intercept         b  r_sqared
0  -0.012392  0.924480  0.002418
1  -0.014309 -0.661212  0.000692
2   0.032170  3.097655  0.014185
3   0.023588  3.619913  0.042710
4  -0.004627 -7.807699  0.222902
5   0.022653  4.237040  0.059848


**comment:** Output has four tables, and each of them represents maturities n = 2, 3, 4, 5. Rows represent subintervals in such a way that 0 (first 120 obs), 1(from 120 obs to 240) and so on. 

For the first 3 rows (subintervals) $R^2$ measure (goodness of fit) declines as we move from maturity n to the maturity n+1, and this decline in predictability of Fama-Bliss model is more pronounced for short term bonds. For the rows indexed with 3 and 4 $R^2$ increases as we move from maturity n to the maturity n+1. For the last interval (column 5) we had lower number of observations (less than 120) so it might no be equally representative, and no clear trend is seen there.

In short, we see a decline in predictability as maturity increases, but no clear decrease in predictability is seen over the time period form 1964 to 2021, except maybe that maximal predictability is achieved during 1974M1–1983M12, 1984M1–1993M12 and 1994M1–2003M12 periods.

> (b.2) **(6 pts)** For the same set of subsamples, as well as for the full sample, run the following complementary regression:
$$y_{t+n−1}^{(1)}−y_{t}^{(1)}={α}^{(n)} + {β}^{(n)}(f_{t}^{(n)}−y_{t}^{(1)})+{η}_{t+n−1}^{(n)}$$
Create the same output tables as for the excess return regressions. Compare
the results of the excess return and complementary regressions. Discuss your
findings.



In [30]:
step = 12
step2 =120
tables = []

for col in range(0,4):
  table = []
  Y1=y.iloc[step*(col+1):,0].values-y.iloc[:-step*(col+1),0].values
  for row in range(1,Y1.shape[0]//step2+1):
    Y11=Y1[(row-1)*step2:row*step2]
    X11=forward_rates_df.iloc[(row-1)*step2:row*step2,col].values-y.iloc[(row-1)*step2:row*step2,col].values

    lm = LinearRegression()
    lm.fit(X11.reshape(X11.shape[0],1), Y11)
    res1 = (lm.intercept_, lm.coef_[0], lm.score(X11.reshape(X11.shape[0],1), Y11))
    table.append(np.array(res1))
    #print(row*step, row)
  
  last = (Y1.shape[0]//step2)*step2
  Y11=Y1[last:last+(Y1.shape[0]%step2)]#rx.iloc[last:last+(rx.shape[0]%step),col].values-y.iloc[last:last+(rx.shape[0]%step),col].values
  X11=forward_rates_df.iloc[last:last+(Y1.shape[0]%step2),col].values-y.iloc[last:last+(Y1.shape[0]%step2),col].values

  lm = LinearRegression()
  lm.fit(X11.reshape(X11.shape[0],1), Y11)
  res1 = (lm.intercept_, lm.coef_[0], lm.score(X11.reshape(X11.shape[0],1), Y11))
  table.append(np.array(res1))

  tables.append(np.array(table))

  
#subintervals
dfs2 = [pd.DataFrame(table, columns=['intercept', 'beta', 'r_sqared']) for table in tables]
for df in dfs2:
  print(df)

   intercept      beta  r_sqared
0   0.004152  0.179033  0.006408
1   0.001295  0.443059  0.029929
2  -0.003907 -0.148323  0.002748
3  -0.006837  0.574220  0.065636
4  -0.006502  1.461208  0.357178
5  -0.005942  1.816015  0.436461
   intercept      beta  r_sqared
0   0.006703  2.483900  0.162990
1   0.002292  1.422864  0.041935
2  -0.016186  1.846756  0.044281
3  -0.011348  1.970079  0.086655
4  -0.014563  5.835930  0.564986
5  -0.003716  3.205340  0.073397
   intercept      beta  r_sqared
0   0.009446  2.497047  0.089955
1   0.001617  3.395236  0.061788
2  -0.006813 -1.246918  0.007633
3  -0.006635  0.842108  0.004055
4  -0.005570  6.336264  0.145093
5   0.009308  1.646609  0.010407
   intercept      beta  r_sqared
0   0.009085  0.821724  0.013871
1   0.006352 -0.975064  0.003159
2  -0.019518 -7.678851  0.464613
3  -0.016624 -7.222796  0.430771
4  -0.027620 -4.904244  0.272434
5  -0.006844 -5.326026  0.487942


In [31]:
#full sample

#very poor fit for the first maturity
Y1=y.iloc[12:,0].values-y.iloc[:-12,0].values

X11=forward_rates_df.iloc[:-12,0].values-y.iloc[:-12,0].values

X11=sm.add_constant(X11)
sm.OLS(Y1,X11).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     5.324
Date:                Sun, 07 Aug 2022   Prob (F-statistic):             0.0213
Time:                        19:51:25   Log-Likelihood:                 1828.5
No. Observations:                 672   AIC:                            -3653.
Df Residuals:                     670   BIC:                            -3644.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.001     -2.069      0.039      -0.003   -7.58e-05
x1             0.2054      0.089      2.307      0.021       0.031       0.380
==============================================================================
Omnibus:                       17.961   Durbin-Watson:                   0.172
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.820
Skew:                          -0.065   Prob(JB):                     1.01e-08
Kurtosis:                       4.139   Cond. No.                         145.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
#second maturity

Y2=y.iloc[24:,1].values-y.iloc[:-24,1].values

X21=forward_rates_df.iloc[:-24,1].values-y.iloc[:-24,1].values

X21=sm.add_constant(X21)
sm.OLS(Y2,X21).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.319
Date:                Sun, 07 Aug 2022   Prob (F-statistic):              0.128
Time:                        19:51:25   Log-Likelihood:                 1618.9
No. Observations:                 660   AIC:                            -3234.
Df Residuals:                     658   BIC:                            -3225.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0018      0.001     -1.904      0.057      -0.004    5.66e-05
x1             0.3577      0.235      1.523      0.128      -0.103       0.819
==============================================================================
Omnibus:                        0.889   Durbin-Watson:                   0.079
Prob(Omnibus):                  0.641   Jarque-Bera (JB):                0.722
Skew:                          -0.047   Prob(JB):                        0.697
Kurtosis:                       3.133   Cond. No.                         289.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
#third maturity

Y3=y.iloc[36:,2].values-y.iloc[:-36,2].values

X31=forward_rates_df.iloc[:-36,2].values-y.iloc[:-36,2].values

X31=sm.add_constant(X31)
sm.OLS(Y3,X31).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2280
Date:                Sun, 07 Aug 2022   Prob (F-statistic):              0.633
Time:                        19:51:25   Log-Likelihood:                 1551.4
No. Observations:                 648   AIC:                            -3099.
Df Residuals:                     646   BIC:                            -3090.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.001     -1.741      0.082      -0.003       0.000
x1            -0.1877      0.393     -0.477      0.633      -0.960       0.584
==============================================================================
Omnibus:                       10.549   Durbin-Watson:                   0.067
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.890
Skew:                           0.270   Prob(JB):                      0.00432
Kurtosis:                       3.335   Cond. No.                         453.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
#fourth maturity

Y4=y.iloc[48:,3].values-y.iloc[:-48,3].values

X41=forward_rates_df.iloc[:-48,3].values-y.iloc[:-48,3].values

X41=sm.add_constant(X41)
sm.OLS(Y4,X41).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     13.45
Date:                Sun, 07 Aug 2022   Prob (F-statistic):           0.000265
Time:                        19:51:26   Log-Likelihood:                 1493.4
No. Observations:                 636   AIC:                            -2983.
Df Residuals:                     634   BIC:                            -2974.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0041      0.001     -3.859      0.000      -0.006      -0.002
x1            -1.3075      0.356     -3.668      0.000      -2.008      -0.608
==============================================================================
Omnibus:                       58.193   Durbin-Watson:                   0.072
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.467
Skew:                           0.634   Prob(JB):                     8.34e-21
Kurtosis:                       4.371   Cond. No.                         388.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

(c) **(3 pts)** If you run the regressions correctly you should see that the slope coefficients add up to one for n = 2, but not for n > 2. Provide a theoretical derivation of this result for n = 2, and discuss why it does not hold for n > 2. 

In [35]:
a=dfs2[0]['beta'].values

In [36]:
b=dfs[0]['b'].values

In [37]:
a+b #it should amount to 1

array([1., 1., 1., 1., 1., 1.])

If the $n=2$ for the forward spot spread, the following identity holds: 

$$f_{t}^{(2)} - y_{t}^{(n)} = p_{t}^{(1)} - p_{t}^{(2)} + p_{t}^{(n)} =$$

$$ = (p_{t}^{(1)} - p_{t}^{(2)} + p_{t}^{(1)}) + (-p_{t+ 1}^{(1)}  + p_{t}^{(1)}) =$$

$$ = (r_{t+n}^{(2)} - y_{t}^{(1)}) + (y_{t+1}^{(1)} - y_{t}^{(1)}) $$ 

out of which we have following two regressions: 

$$ r_{t+1}^{(2)} - y_{t}^{(n)} = b  \cdot (f_{t}^{(2)} - y_{t}^{(1)})$$

$$ y_{t+1}^{(1)} - y_{t}^{(1)} = 	\beta  \cdot (f_{t}^{(2)} - y_{t}^{(1)})$$

In simple linear regression removal of the intercept is not affecting the value of slope, which is why we didn't write intercepts $\alpha$ and $\alpha$.

$$r_{t+1}^{(2)} - y_{t}^{(n)} = b  \cdot (r_{t+1}^{(2)} - y_{t}^{(n)}) + (y_{t+1}^{(1)} - y_{t}^{(1)})$$

$$y_{t+1}^{(1)} - y_{t}^{(1)} = 	\beta  \cdot (r_{t+1}^{(2)} - y_{t}^{(1)}) + (y_{t+1}^{(1)} - y_{t}^{(1)})$$

$$[(r_{t+1}^{(2)} - y_{t}^{(n)}) + (y_{t+1}^{(1)} - y_{t}^{(1)})] = (b+\beta)  \cdot [(r_{t+1}^{(2)} - y_{t}^{(n)}) + (y_{t+1}^{(1)} - y_{t}^{(n)})] $$

$$[b+\beta-1] \cdot[(r(2)t+1−y(n)t)+(y(1)t+1−y(1)t)] = 0$$
$$b+\beta -1 = 0$$
$$b+\beta = 1$$
for $n\neq 2$ the identity for forward spread is not valid, so $b + \beta = 1$ for $n=2$

### **4 Fixed Income Derivatives (8 pts)**

Consider a six-month plain vanilla swap contract with the maturity 5 years, the notional amount of $1 million, and the fixed leg rate of 5%. Furthermore, assume the following cash flow timeline:

|      |     | $F$ |     |  $F$  |     | $F$  |     | $F$ |     | $F$  |
|-------:|-------:|-------:|-------:|-------:|-------:|-----:|-----:|-----:|-----:|-----:|
| $T_{0}$ | $T_{1}$  | $T_{2}$  | $T_{3}$  | $T_{4}$  | $T_{5}$  | $T_{6}$  | $T_{7}$  | $T_{8}$  | $T_{9}$  | $T_{10}$ |
|       | $-V_{0}$ | $-V_{1}$ | $-V_{2}$ | $-V_{3}$ | $-V_{4}$ | $-V_{5}$ | $-V_{6}$ | $-V_{7}$ | $-V_{8}$ | $-V_{9}$ |

Note that $T_{i+1} − T_i$ = 6 months ($i$ = 0, 2, . . . , 9).

At time $T_0$ (i.e., today), the spot rate curve is given by:

| Maturity (Years) | Spot Rate (%) |
|:----------------:|:-------------:|
|        $0.5$       |     $3.005$     |
|        $1.0$       |     $3.575$     |
|        $1.5$       |     $3.925$     |
|        $2.0$       |     $4.134$     |
|        $2.5$       |     $4.412$     |
|        $3.0$       |     $4.499$     |
|        $3.5$       |     $4.785$     |
|        $4.0$       |     $4.896$     |
|        $4.5$       |     $5.001$     |
|        $5.0$       |     $5.069$     |

(a) **(1 pts)** What is the pricing formula for this plain vanilla swap using the zero-coupon method?


$$PVS=LegPrice_{fixed}-LegPrice_{floating}$$

(b) **(3 pts)** Calculate the swap price. 

In [38]:
#calculating the fixed leg price
#importing the necesseray packages
import sympy as sp
import numpy as np

#defining symbols
r=sp.Symbol('r')
FV=sp.Symbol('FV')
y=sp.Symbol('y')
y1,y2,y3,y4,y5,y6,y7,y8,y9,y10=sp.symbols('y1,y2,y3,y4,y5,y6,y7,y8,y9,y10')

#defining equation  and substituting symobols
Fixed_leg_price=(sum(r/(1+np.array([y1,y2,y3,y4,y5,y6,y7,y8,y9]))**np.arange(0.5,5,0.5))+(FV+r)/(1+y10)**5).subs(dict(y1=0.03005,y2=0.03575,y3=0.03925,y4=0.04134,y5=0.04412,y6=0.04499,y7=0.04785,y8=0.04896,y9=0.05001,y10=0.05069,r=25000,FV=1000000))

Fixed_leg_price

1002199.73629166

In [39]:
#in order to calculate floating leg we need to calculate forward rate between 0.5 and 1

#defing symbols 
s1=sp.Symbol('s1')
f1=sp.Symbol('f1')
s2=sp.Symbol('s2')

#calculating forward rate based on no arbitrage
fr=sp.solve(sp.Eq(lhs=1/(1+s1)**0.5*1/(1+f1)**0.5-1/(1+s2),rhs=0),f1)[0].subs(dict(s2=3.575/100,s1=3.005/100))#proof of no arbitrage

#calculating floating price
Floating_price=(1000000*(1+fr)/((1+y)**0.5)).subs(y,3.005/100)
Floating_price

1026177.34924117

In [40]:
swap_price=Fixed_leg_price-Floating_price
swap_price

-23977.6129495077

**comment:** The swap value is negative, which means that the decrease in the value of the fixed cash flow is higher than the decrease in the value of the floating cash flow.

(c) **(1 pts)** What is the swap rate such that the price of this swap is zero at time $T_0$? 

In [41]:
absolute_value_semmi_annual=sp.solve((sum(r/(1+np.array([y1,y2,y3,y4,y5,y6,y7,y8,y9]))**np.arange(0.5,5,0.5))+(FV+r)/(1+y10)**5-Floating_price).subs(dict(y1=0.03005,y2=0.03575,y3=0.03925,y4=0.04134,y5=0.04412,y6=0.04499,y7=0.04785,y8=0.04896,y9=0.05001,y10=0.05069,FV=1000000)),r)[0]

(absolute_value_semmi_annual*2/1000000)*100 #swap rate in percentage

5.54188423743899

(d) **(3 pts)** Suppose you purchased $10 million worth of a bond with the tenor of four years, and the coupon rate of 6.2% (coupons are paid out annually). How many swaps do you have to buy or sell to protect your bond from adverse interest rate moves? 

In [42]:
#First dollar duration of the swap is calculated by dd_fixed-dd_float

FL_dollar_duration=sp.diff(sum(r/(1+y)**np.arange(0.5,5,0.5))+((FV+r)/(1+y)**5),y).subs(dict(y=0.0495,FV=1000000,r=25000))
dollar_duration_float=sp.diff((1000000*(1+fr)/(1+y)**0.5),y).subs(y,sp.solve((1000000*(1+fr)/(1+y))-Floating_price,y)[0])


dd_sto_nam_treba=FL_dollar_duration-dollar_duration_float


dd_bond=sp.diff(sum(r/(1+np.array([y,y,y]))**np.arange(1,4,1))+(FV+r)/(1+y)**4,y).subs(dict(r=620000,y=0.0484,FV=10000000))

In [43]:
#We asume that we can buy fraction of bond.
-dd_bond/dd_sto_nam_treba


-9.69702871334731

**comment**:We would need to sell 9.69 swaps to be hedged with linear aproximation (first derivative in Taylor row) against adverse interest rates.

### **5 Stochastic Models of Interest Rates (10 pts)**

The Vasiček model is a popular one-factor short-rate model. In this exercise, we will consider a calibration of the Vasiček model, and its application for bond pricing.

Vasiček (1977) specified the short-rate dynamics in the following form:
$$ dr_t = a(b − r_t)dt + {\sigma}dW_t$$

where $a$ and $b$ represent the speed and level of mean reversion, respectively. The parameter ${\sigma}$ is the volatility of the short rate, and $r_0$ is the instantaneous short rate at time $t = 0$. The stochastic nature of the short rate process—the so-called Ornstein–Uhlenbeck process—is captured by a standard Brownian motion $\left\{W_t\right\}_{t≥0}$, defined on a filtered probability space
$(\Omega,F, \mathbb{F} = \left\{F_t, t \ge 0\right\}, \mathbb{P})$ which satisfies the usual conditions.

(a) **(5 pts)** Consider the data provided in in the Excel sheet **Yield Curve**. Using the expression for zero-coupon yields in the Vasiček model $y_{0,t}^{Mdl}$ (see the lecture slides), estimate the model parameters $Θ = {r_0, a, b, σ}$ for each observation date separately by minimizing the following objective function:
$$\min_{Θ}\sum_T(y_{0,T}^{Mdl}− y_{0,T}^{Mkt})^2,$$
where $y_{0,t}^{Mkt}$ are market spot rates for the available maturities $T$. Note that this approach makes the model parameters dependent on time, hence it can be referred to as the dynamic Vasiček model.
Create a table which reports $\hat{r_{0}}, \hat{a}, \hat{b}, \hat{σ}$, as well as the mean squared error (MSE), i.e., the row names should be observation dates, whereas the column dates should be the model coefficients.
<p><u>Hint</u>: Depending on the optimization routine used, you might want to impose certain constraints in your minimization procedure (e.g., volatility should be positive, mean reversion level should be withing a reasonable interest rate range, etc). Alternatively, you can consider a combination of OLS regressions and grid search (similar to the procedure described in Problem 2). Recall that the Vasiček zero-coupon bond yield can be expressed as the sum of a constant term, a term proportional to $g(a, T)$, and a term proportional to $g(2a, T)$, where $$ g(ξ, T) := \frac{1 − e^{−ξT}}{ξT}.$$ </p>
<p>For a given observation date and a fixed speed of mean reversion a, you can run a regression of market spot rates on $\left\{1, g(a, T), g(2a, T)\right\}$. All other steps are—in principle—the same as for the Nelson–Siegel model for yield curve fitting. You only have to come up with a reasonable range of values for $r_0$.</p>

In [44]:
df=pd.read_excel('MCF22_FIC_Final_Exam_Group_7.xlsx').iloc[:,1:]

df#loading the df just in case

,0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30
0,2.4576,2.7223,2.9727,3.2573,3.4251,3.5557,3.6788,3.8022,3.9259,4.0473,4.1633,4.2716,4.6617,4.8054,4.7696,4.6217
1,2.7450,2.9724,3.2540,3.5845,3.7567,3.8734,3.9757,4.0770,4.1797,4.2817,4.3804,4.4732,4.8040,4.9051,4.8358,4.6617
2,2.7664,3.1196,3.4287,3.7855,3.9514,4.0541,4.1429,4.2334,4.3275,4.4227,4.5156,4.6031,4.9084,4.9827,4.8879,4.6916
3,2.8891,3.1711,3.3855,3.6232,3.7250,3.7923,3.8608,3.9392,4.0260,4.1169,4.2076,4.2946,4.6152,4.7222,4.6663,4.5082
4,2.9398,3.1078,3.3845,3.5692,3.6303,3.6672,3.7129,3.7738,3.8468,3.9268,4.0087,4.0887,4.3919,4.5008,4.4567,4.3148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0.0482,0.0659,0.0998,0.1132,0.2025,0.3317,0.4779,0.6268,0.7701,0.9034,1.0246,1.1332,1.5152,1.7319,1.8850,2.0213
193,0.0330,0.0456,0.0956,0.1654,0.3433,0.5569,0.7704,0.9677,1.1429,1.2956,1.4273,1.5406,1.9163,2.1193,2.2459,2.3332
194,0.0152,0.0304,0.0747,0.1683,0.3967,0.6694,0.9395,1.1854,1.3988,1.5790,1.7284,1.8510,2.2016,2.3577,2.4732,2.5936
195,0.0025,0.0228,0.0742,0.1684,0.3779,0.6275,0.8768,1.1060,1.3072,1.4786,1.6219,1.7404,2.0833,2.2388,2.3584,2.4882


In [45]:
#defing the function which is going to be optimized

def Vlasicek1(a,b,r0,sigma,T):
    return b+(r0-b)*((1-math.e**(-a*T))/(a*T))-((sigma**2)/(2*a**2))*(1-2*((1-math.e**(-a*T))/(a*T))+((1-math.e**(-2*a*T))/(2*a*T)))

In [46]:
#performing the optimization 

import scipy.optimize as sco
#defining empty list for storage
parameters=[]#parameters
msfe=[]#msfe
succ=[]#succes(total should be 197)
param=np.ones((4,1))
#performing an optimization
for i in range(df.shape[0]):
    parameters.append(sco.minimize(lambda parametri:sum((df.iloc[i,:]-Vlasicek1(parametri[0],parametri[1],parametri[2],parametri[3],np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30])))**2)/16,param,constraints=[dict(type='ineq',fun=lambda parametri:-parametri[1]+6.5),dict(type='ineq',fun=lambda parametri:parametri[3]-0.1)])['x'])
    msfe.append(sco.minimize(lambda parametri:sum((df.iloc[i,:]-Vlasicek1(parametri[0],parametri[1],parametri[2],parametri[3],np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30])))**2)/16,param,constraints=[dict(type='ineq',fun=lambda parametri:-parametri[1]+6.5),dict(type='ineq',fun=lambda parametri:parametri[3]-0.1)])['fun'])
    succ.append(sco.minimize(lambda parametri:sum((df.iloc[i,:]-Vlasicek1(parametri[0],parametri[1],parametri[2],parametri[3],np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10,15,20,25,30])))**2)/16,param,constraints=[dict(type='ineq',fun=lambda parametri:-parametri[1]+6.5),dict(type='ineq',fun=lambda parametri:parametri[3]-0.1)])['success'])
    param=parameters[-1]#initial guess should be the result of the previous iteration

In [47]:
#check weather every iteration is successfull

sum(succ)==197

True

(b) **(2 pts)** Plot the dynamics of the estimated model parameters over time (three separate plots). Discuss your results. 

In [48]:
#defining dataframe

df_rad_vlasicek=pd.DataFrame(np.array(parameters))


df_rad_vlasicek['msfe']=msfe


df_rad_vlasicek.columns=['a','b','r0','sigma','msfe']
df_rad_vlasicek.index=pd.read_excel('MCF22_FIC_Final_Exam_Group_7.xlsx')['Date']

In [49]:
#ploting parameters
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(y=df_rad_vlasicek['msfe'],x=df_rad_vlasicek.index))

fig.update_layout(
    title="MSFE",
    xaxis_title="Date",
    yaxis_title="Parameter value")

In [50]:
fig=go.Figure()
fig.add_trace(go.Scatter(y=df_rad_vlasicek['a'],x=df_rad_vlasicek.index))

fig.update_layout(
    title="a",
    xaxis_title="Date",
    yaxis_title="Parameter value")

In [51]:
fig=go.Figure()
fig.add_trace(go.Scatter(y=df_rad_vlasicek['b'],x=df_rad_vlasicek.index))


fig.update_layout(
    title="b",
    xaxis_title="Date",
    yaxis_title="Parameter value")


In [52]:
fig=go.Figure()
fig.add_trace(go.Scatter(y=df_rad_vlasicek['r0'],x=df_rad_vlasicek.index))
fig.update_layout(
    title="ro",
    xaxis_title="Date",
    yaxis_title="Parameter value")

**comment:** The graphs clearly show the periods of crisis. Also in the period of 2019 and Covid crisis the MSE is growing (although only 0.0004, but this is due to squared error, which means it is of the same magnitude as the data used to compute it). This is why results of the model do not appear staitstically significant. 

(c) **(3 pts)** Using a set of Vasiček model parameters of your choice, demonstrate that the price of a zero-coupon bond (e.g., with the maturity of one year and the face value of $100), calculated using a Monte Carlo simulation approach converges—with the increasing number of simulations—to the the theoretical bond price provided in the lecture slides.

In [53]:
from numpy import random as npr

In [54]:
# r0,a,b,sigma

def g_function(a, tau):

    return (1 - np.exp(-a*tau))/(a*tau)


def vasicek_yield_model(x, const):
    arr = np.array([])
    for val in const:
        arr = np.append(arr,
        (x[2] - x[3]**2/(2*x[1]**2))
        +(x[0] - x[2] + x[3]**2/x[1]**2)*g_function(x[1],val)
        -x[3]**2/(4*x[1]**2)*g_function(2*x[1],val)
        )
    
    return arr

#2005-01-31	0.025103	0.296	0.050880	0.001588	9.224842e-07 - r0,a,b,sigma
#Zero coupon bond B(0,T)=e^(-y_T*T) where y_T is the continuous time yield for a zero coupon bond with maturity T
vasicek_yield=vasicek_yield_model([0.025103,0.296,0.050880,0.001588], [1])
#vasicek_yield_model definisan je u zadatku pod a)
B = np.exp(-1*vasicek_yield)
B

array([0.97183139])

**Comment:** In vasicek_ZCB_price function we are doing integration over one path and then we do mean ( expected value ) over simulations.
$$B(0,t)=E_0[e^{-\int _{0 }^{T }r_tdt}]$$

In [55]:
def vasicek_ZCB_price(r, T,M):
    #vasicek_ZCB_price - integral estimate
    #
    
    return np.exp(-np.sum(r, axis = 0)*T/M).mean()

In [56]:
M0 = 200
T0=1
def mc_vasicek_euler(r0=0.025103,a=0.296,b=0.050880,T=1,sigma=0.001588,I=10000,M=200):
    """Simulates I random paths with M time steps with Euler scheme. Parameters are:
    - r0: initial level of interest rates
    - a: speed of convergion toward log term level of interest rates
    - b: long term level of interest rates
    - sigma: volatility of interest rates
    - I: number of simulations
    - M: number of steps in each simulation"""
    dt = T / M
    rh = np.zeros((M + 1, I))
    rh[0] = r0
    for t in range(1, M + 1):
        rh[t]=np.maximum( rh[t-1]+a*(b - rh[t-1])*dt+sigma*np.sqrt(dt)*npr.standard_normal(I), 0)
    return rh

def calculate_vasicek_ZCB_price(T,I,M):
    #calculate_vasicek_ZCB_price
    r= mc_vasicek_euler(r0=0.025103,a=0.296,b=0.050880,T=1,sigma=0.001588,I=10000,M=200)
    return vasicek_ZCB_price(r, T,M)


In [57]:
#difference between closed solution for I = 1000
calculate_vasicek_ZCB_price(T=1,I=1000,M=200)-B

array([-0.00014024])

In [58]:
#difference between closed solution for I = 50000
calculate_vasicek_ZCB_price(T=1,I=50000,M=200)-B

array([-0.00012655])

In [59]:
#difference between closed solution for I = 100000
calculate_vasicek_ZCB_price(T=1,I=100000,M=200)-B

array([-0.00012095])

In [60]:
#difference between closed solution for I = 500000
calculate_vasicek_ZCB_price(T=1,I=500000,M=200)-B

array([-0.00013626])

In [61]:
#difference between closed solution for I = 1000000
calculate_vasicek_ZCB_price(T=1,I=100000,M=200)-B

array([-0.00013877])